In [1]:
from pathlib import Path

from sneakers_ml.features.features import save_features
from sneakers_ml.features.resnet152 import get_resnet152_features

/home/seara/.cache/pypoetry/virtualenvs/sneakers-ml-hflTz_mY-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
features, classes, class_to_idx = get_resnet152_features("data/merged/images/by-brands")

100%|██████████| 93/93 [00:50<00:00,  1.84it/s]


In [8]:
save_path = Path("data/features/by-brands/resnet152.pickle")
save_features(save_path, features, classes, class_to_idx)